### Imports

In [323]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import pickle
import re
import numpy as np
import pandas as pd
import string
import os
import nltk
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.collocations import *
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tadiw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the Text 

In [324]:
f=open('the-great-gatsby.txt')
script=[]
script=f.read()

In [325]:
from nltk.tokenize import word_tokenize
book = word_tokenize(script)
book[:10]

['The',
 'Great',
 'Gatsby',
 'By',
 'F.',
 'Scott',
 'Fitzgerald',
 '.',
 'Then',
 'wear']

In [326]:
stopwords_list = stopwords.words('english') + list(string.punctuation) + ['“','”','...',"''",'’','``', "'", "‘"]
custom_stopwords = ['–', 'also', 'something', 'cf', 'thus', 'two', 'now', 'would', 'make', 'eb', 'u', 'well', 'even', 'said', 'eg', 'us',
                    'n', 'sein', 'e', 'da', 'therefore', 'however', 'would', 'thing', 'must', 'merely', 'way', 'since', 'latter', 'first',
                    'B', 'mean', 'upon', 'yet', 'cannot', 'c', 'C', 'let', 'may', 'might', "'s", 'b', 'ofthe', 'p.', '_', '-', 'eg', 'e.g.',
                    'ie', 'i.e.', 'f', 'l', "n't", 'e.g', 'i.e', '—', '--', 'hyl', 'phil', 'one', 'press', 'cent', 'place'] + stopwords_list

In [327]:
cleaned_words = [x.lower() for x in book if x.lower() not in custom_stopwords]
freq_dist = FreqDist(cleaned_words)

In [328]:
uninformative_words = ['else', 'shall', 'either', 'still', 'rather', 'another', 'made', 'without']
book_stopwords = custom_stopwords + [x[0] for x in freq_dist.most_common(50)] + uninformative_words

In [329]:
book

['The',
 'Great',
 'Gatsby',
 'By',
 'F.',
 'Scott',
 'Fitzgerald',
 '.',
 'Then',
 'wear',
 'the',
 'gold',
 'hat',
 ',',
 'if',
 'that',
 'will',
 'move',
 'her',
 ';',
 'If',
 'you',
 'can',
 'bounce',
 'high',
 ',',
 'bounce',
 'for',
 'her',
 'too',
 ',',
 'Till',
 'she',
 'cry',
 'â€˜Lover',
 ',',
 'gold-hatted',
 ',',
 'high-bouncing',
 'lover',
 ',',
 'I',
 'must',
 'have',
 'you',
 '!',
 'â€™',
 'â€',
 '”',
 'THOMAS',
 'PARKE',
 'Dâ€™INVILLIERS',
 '\x18',
 'The',
 'Great',
 'Gatsby',
 'Chapter',
 '1',
 'I',
 'n',
 'my',
 'younger',
 'and',
 'more',
 'vulnerable',
 'years',
 'my',
 'father',
 'gave',
 'me',
 'some',
 'advice',
 'that',
 'Iâ€™ve',
 'been',
 'turning',
 'over',
 'in',
 'my',
 'mind',
 'ever',
 'since',
 '.',
 'â€˜Whenever',
 'you',
 'feel',
 'like',
 'criticizing',
 'any',
 'one',
 ',',
 'â€™',
 'he',
 'told',
 'me',
 ',',
 'â€˜just',
 'remember',
 'that',
 'all',
 'the',
 'people',
 'in',
 'this',
 'world',
 'havenâ€™t',
 'had',
 'the',
 'advantages',
 'that',
 

### Deep Cleaning

In [330]:
def baseline_clean(text):
    # remove utf8 encoding characters and some punctuations
    text = [re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff\xad\x0c6§\\\£\Â*_<>""⎫•{}Γ~]', ' ', word) for word in text]
    text = [word for word in text if word.isalpha()]
  
    #removing all punctuation
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
    text = [re_punc.sub( '' , word) for word in text]
    
    # replace whitespace characters with actual whitespace
    text = [re.sub(r'\s', ' ',word)for word in text]

    # remove or standardize some recurring common and meaninless words/phrases/characters
    text = [re.sub(r'\s*â€\s*', ' ', word)for word in text]

    # replace odd quotation marks with a standard
    text = [re.sub(r'[‘’“”]', "'", word)for word in text]


    # unify some abbreviations
    text = [re.sub(r'&', 'and', word)for word in text]
    text = [re.sub(r'\se\.g\.\s', ' eg ', word)for word in text]
    text = [re.sub(r'\si\.e\.\s', ' ie ', word)for word in text]
    text = [re.sub('coroll\.', 'coroll', word)for word in text]
    text = [re.sub('pt\.', 'pt', word)for word in text]

    # remove roman numerals, first capitalized ones
    text = [re.sub(r'\s((I{2,}V*X*\.*)|(IV\.*)|(IX\.*)|(V\.*)|(V+I*\.*)|(X+L*V*I*]\.*))\s', ' ', word) for word in text]
    # then lowercase
    text = [re.sub(r'\s((i{2,}v*x*\.*)|(iv\.*)|(ix\.*)|(v\.*)|(v+i*\.*)|(x+l*v*i*\.*))\s', ' ', word) for word in text]

    # remove periods and commas flanked by numbers
    text = [re.sub(r'\d\.\d', ' ', word)for word in text]
    text = [re.sub(r'\d,\d', ' ', word)for word in text]

    # remove the number-letter-number pattern used for many citations
    text = [re.sub(r'\d*\w{,2}\d', ' ', word)for word in text]

    # remove numerical characters
    text = [re.sub(r'\d+', ' ', word)for word in text]


    # remove isolated colons and semicolons that result from removal of titles
    text = [re.sub(r'\s+:\s*', ' ', word)for word in text]
    text = [re.sub(r'\s+;\s*', ' ', word)for word in text]

    # remove isolated letters (do it several times because strings of isolated letters do not get captured properly)
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    text = [re.sub(r'\s[^aAI\.]\s', ' ', word)for word in text]
    
    # remove isolated letters at the end of sentences or before commas
    text = [re.sub(r'\s[^aI]\.', '.',word)for word in text]
    text = [re.sub(r'\s[^aI],', ',', word)for word in text]

    # deal with spaces around periods and commas
    text = [re.sub(r'\s+,\s+', ', ', word)for word in text]
    text = [re.sub(r'\s+\.\s+', '. ', word)for word in text]

    # remove empty parantheses
    text = [re.sub(r'(\(\s*\.*\s*\))|(\(\s*,*\s*)\)', ' ', word)for word in text]

    # reduce multiple periods, commas, or whitespaces into a single one
    text = [re.sub(r'\.+', '.', word)for word in text]
    text = [re.sub(r',+', ',', word)for word in text]
    text = [re.sub(r'\s+', ' ', word)for word in text]

    return text

In [331]:
txt=baseline_clean(book)

In [332]:
txt

['The',
 'Great',
 'Gatsby',
 'By',
 'Scott',
 'Fitzgerald',
 'Then',
 'wear',
 'the',
 'gold',
 'hat',
 'if',
 'that',
 'will',
 'move',
 'her',
 'If',
 'you',
 'can',
 'bounce',
 'high',
 'bounce',
 'for',
 'her',
 'too',
 'Till',
 'she',
 'cry',
 'lover',
 'I',
 'must',
 'have',
 'you',
 'THOMAS',
 'PARKE',
 'The',
 'Great',
 'Gatsby',
 'Chapter',
 'I',
 'n',
 'my',
 'younger',
 'and',
 'more',
 'vulnerable',
 'years',
 'my',
 'father',
 'gave',
 'me',
 'some',
 'advice',
 'that',
 'been',
 'turning',
 'over',
 'in',
 'my',
 'mind',
 'ever',
 'since',
 'you',
 'feel',
 'like',
 'criticizing',
 'any',
 'one',
 'he',
 'told',
 'me',
 'remember',
 'that',
 'all',
 'the',
 'people',
 'in',
 'this',
 'world',
 'had',
 'the',
 'advantages',
 'that',
 'He',
 'say',
 'any',
 'more',
 'but',
 'always',
 'been',
 'unusually',
 'communicative',
 'in',
 'a',
 'reserved',
 'way',
 'and',
 'I',
 'understood',
 'that',
 'he',
 'meant',
 'a',
 'great',
 'deal',
 'more',
 'than',
 'that',
 'In',
 'c

In [333]:
# organize into sequences of tokens
length = 5 + 1
sequences = list()
for i in range(length, len(txt)):
# select sequence of tokens
    seq = txt[i-length:i]
# convert into a line
    line = " ".join(seq)
# store
    sequences.append(line)
print( ' Total Sequences: %d ' % len(sequences))


 Total Sequences: 44710 


In [334]:
sequences[0]

'The Great Gatsby By Scott Fitzgerald'

In [335]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequence_data = tokenizer.texts_to_sequences(sequences)

sequence_data = np.array(sequence_data,dtype=object)

In [336]:
X, y = sequence_data[:, :-1], sequence_data[:,-1]

In [337]:
X[0], y[0]

(array([1, 213, 32, 50, 5548], dtype=object), 2555)

In [338]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5549


In [339]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [[1 213 32 50 5548]
 [213 32 50 5548 2555]
 [32 50 5548 2555 44]
 [50 5548 2555 44 1293]
 [5548 2555 44 1293 1]]
The responses are:  [2555 44 1293 1 619]


In [340]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [341]:
sequence_length = X.shape[1]
sequence_length

5

### Creating the Model:

In [342]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=sequence_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [343]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 5, 10)             55490     
                                                                 
 lstm_22 (LSTM)              (None, 5, 100)            44400     
                                                                 
 lstm_23 (LSTM)              (None, 100)               80400     
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 5549)              560449    
                                                                 
Total params: 750,839
Trainable params: 750,839
Non-trainable params: 0
_________________________________________________________________


In [344]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'

In [345]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

In [346]:
X=np.asarray(X).astype('float32')
y=np.asarray(y).astype('float32')

In [347]:
model.fit(X, y,batch_size=128, epochs=100, verbose=1)

Epoch 1/100
350/350 [==============================] - 71s 79ms/step - loss: 6.7920
Epoch 2/100
350/350 [==============================] - 22s 62ms/step - loss: 6.4546
Epoch 3/100
350/350 [==============================] - 15s 43ms/step - loss: 6.4301
Epoch 4/100
350/350 [==============================] - 13s 38ms/step - loss: 6.3457
Epoch 5/100
350/350 [==============================] - 15s 42ms/step - loss: 6.2486
Epoch 6/100
350/350 [==============================] - 12s 34ms/step - loss: 6.0890
Epoch 7/100
350/350 [==============================] - 12s 33ms/step - loss: 5.9195
Epoch 8/100
350/350 [==============================] - 14s 41ms/step - loss: 5.8053
Epoch 9/100
350/350 [==============================] - 13s 38ms/step - loss: 5.7021
Epoch 10/100
350/350 [==============================] - 12s 35ms/step - loss: 5.5951
Epoch 11/100
350/350 [==============================] - 14s 40ms/step - loss: 5.4943
Epoch 12/100
350/350 [==============================] - 15s 42ms/step - lo

350/350 [==============================] - 10s 29ms/step - loss: 1.6016
Epoch 98/100
350/350 [==============================] - 10s 29ms/step - loss: 1.5871
Epoch 99/100
350/350 [==============================] - 10s 29ms/step - loss: 1.5569
Epoch 100/100
350/350 [==============================] - 10s 29ms/step - loss: 1.5511


In [350]:
score = model.evaluate(X, y, verbose=0)
string_score=str(score)
print('Test loss:', string_score[0])
print('Test accuracy:', string_score[1])

Test loss: 1
Test accuracy: .


### Prediction

In [351]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword1.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, germ_text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(1):
        encoded = tokenizer.texts_to_sequences([germ_text])[0]
        padded_sequences=pad_sequences([encoded], sequence_length, truncating="pre")
        
        print(encoded, padded_sequences)
        for i in (model.predict(padded_sequences)[0]).argsort()[-5:][::-1]:
          predicted_word = tokenizer.index_word[i]
          print("Next word suggestion:",predicted_word)

In [352]:
germ_text = sequences[110]
germ_text

'inclined to reserve all judgments a'

In [353]:
Predict_Next_Words(model, tokenizer,  germ_text)

[2963, 5, 1955, 32, 1956, 3] [[   5 1955   32 1956    3]]
Next word suggestion: afternoon
Next word suggestion: are
Next word suggestion: never
Next word suggestion: him
Next word suggestion: is


In [354]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "At nine o’clock"
# text2 = "He was balancing"
# text3 = "I’m going to tell "
# text4 = "Most of the confidences are"
# text5 = "stop the script"



while(True):

    text = input("Enter your line: ")
    
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.strip().lower()
            encoded_text = tokenizer.texts_to_sequences([text])[0]
            print(encoded_text)
            pad_encoded = pad_sequences([encoded_text], sequence_length, truncating='pre')
            print(encoded_text, pad_encoded)
            for i in (model.predict(pad_encoded)[0]).argsort()[-5:][::-1]:
                predicted_word = tokenizer.index_word[i]
                print("Next word suggestion:",predicted_word)
                
                
            
        except:
            continue

Enter your line: inclined to reserve all judgments a
[2963, 5, 1955, 32, 1956, 3]
[2963, 5, 1955, 32, 1956, 3] [[   5 1955   32 1956    3]]
Next word suggestion: afternoon
Next word suggestion: are
Next word suggestion: never
Next word suggestion: him
Next word suggestion: is
Enter your line: stop the script
Ending The Program.....


## Run the following code in terminal to view the streamlit app attached with my submission
### streamlit run app.py